### Hansen model with divisible labor by using PEA

#### This code solves business cycle model with AR(1) shocks following Hansen (1985) by using PEA. The model includes both labor choice and uncertainty. We detrend data by applying the HP filter and then compare summary statistics between simulation and Hansen's.


##### Translated from Eva Carceles-Poveda's MATLAB codes

In [1]:
# Importing packages
import numpy as np

# This is used for the root_scalar function
import scipy.optimize as opt

# This is used to calculate the excution time of the while loop
import time

# needed for compact printing of numpy arrays
# use precision to set the number of decimal digits to display
# use suppress=True to show values in full decimals instead of using scientific notation
np.set_printoptions(suppress=True,precision=4,linewidth=np.inf)

#### The nlls (Non-linear least squares) function is copied below

In [2]:
def nlls(DEP,IND1,IND2,T,IBETA):
    
    GBETA = np.zeros((3,1))

    HALT = 0

    DER1 = np.zeros((T-1,1))
    DER2 = np.zeros((T-1,1))
    DER3 = np.zeros((T-1,1))
    F = np.zeros((T-1,1))

    MAX_IT = 100
    smooth = 0.6
    prec = 0.001

    it = 0

    while (HALT == 0 and it < MAX_IT):

        # TAYLOR AROUND IBETA

        IBETA = (smooth*IBETA) + (1-smooth)*GBETA


        DER1 = np.exp(IBETA[0]*np.ones((T-1,1))+(IBETA[1]*IND1)+(IBETA[2]*IND2))
        F = DER1.copy()
        DER2 = F * IND1
        DER3 = F * IND2

        DER = np.concatenate((DER1,DER2,DER3), axis=1)

        # 1.1 Estimate coefficients of linearized equation
        #################################################################
        # DEP - F + DER(Tx3)*IBETA(3x1) = DER(Tx3)*GBETA(3x1) + Error   #
        #################################################################

        Y = DEP - F + np.matmul(DER,IBETA)

        GBETA = np.linalg.inv(DER.T@DER)@DER.T@Y;
        diffnlls = np.matmul(np.ones((1,3)),np.abs(GBETA-IBETA))

        if diffnlls < prec:
            HALT = 1

        it = it + 1

    return GBETA

#### The Hodrick-Prescott filter function is copied below

In [3]:
def hp1(y,w):

    """ Syntax: yhp, ytr = hp1(y, w)
    
    This function computes the filtered series of y, using
    a smoothing parameter w. 
    
    The code is from I. Izvorski.
    """

    t, s = y.shape
    
    if t < s:
        y = y.T

    a = 6*w + 1
    b = -4*w
    c = w
    d = np.array([[c,b,a]])
    d = np.ones((t,1))*d
    m = np.diag(d[:,2])+np.diag(d[0:-1,1],1)+np.diag(d[0:-1,1],-1)
    m = m+np.diag(d[0:-2,0],2)+np.diag(d[0:-2,0],-2)

    m[0,0] = 1+w;       m[0,1] = -2*w;
    m[1,0] = -2*w;      m[1,1] = 5*w+1;
    m[-2,-2] = 5*w+1;   m[-2,-1] = -2*w;
    m[-1,-2] = -2*w;    m[-1,-1] = 1+w;
    
    ytr = np.matmul(np.linalg.inv(m),y)
    yhp = y-ytr

    return yhp, ytr

In [4]:
# Model Parameters
alf = 0.36
beta = 0.99
a = 2
delta = 0.025
rho = 0.95
sigmae = 0.00712
Ez = 1   # expectation of shock

# Labor working hour
lmin = 1e-5
lmax = 1-lmin

# Algorithm Parameters
siyes = 1     # simulte model if equals to 1
stationary=1  # finds stationary distribution for capital
tolv = 1e-4   # tolerance level
lam = 0.75    # updating parameter
T = 10000     # simulation period
loadin = 0    # load results

In [5]:
# Define Steady state variables
xx = (1-beta*(1-delta))/(beta*alf*Ez)
yy = ((1/beta+delta-1)/alf*(1+(1-alf)/a)-delta)*a/((1-alf)*Ez)
l_ss = xx/yy  # for this set of parameters, the steady state labor approxiamately equals to 1/3
k_ss = xx**(1/(alf-1))*l_ss
y_ss = Ez*k_ss**alf*l_ss**(1-alf)
i_ss = delta*k_ss
c_ss = y_ss-i_ss

In [6]:
# Simulation of continuous shocks process
zt = np.ones((T+1,1))

rng = np.random.Generator(np.random.MT19937(0))
for t in np.arange(1,T+1):
    zt[t] = np.exp(rho* np.log(zt[t-1]) + sigmae*rng.standard_normal((1,1)))

#zt = zt.T

In [7]:
if loadin == 1:
    with open('parameters.npy', 'rb') as f:
        bet = np.load(f)
else:
    # initial value of parameters
    bet = np.zeros((3,1))
    bet[0] = (alf*k_ss**(alf-1)*l_ss**(1-alf)+1-delta)/c_ss
    bet0 = np.ones((3,1))
    bet = (bet-lam)/(1-lam)

    # Define matrices
    c = np.zeros((T+1,1))
    k = np.zeros((T+1,1))
    y = np.zeros((T+1,1))
    l = np.zeros((T+1,1))
    inv = np.zeros((T+1,1))
    Pea = np.zeros((T+1,1))

    it = 0
    start = time.perf_counter()
    
    while np.abs(bet-bet0).max() > tolv:
        it = it+1
        bet = lam*bet0+(1-lam)*bet
        bet0 = bet.copy()

        # Simulation for given parameters
        k[0] = k_ss
        c[0] = c_ss
        l[0] = l_ss
        inv[0] = i_ss
        y[0] =  y_ss

        for t in np.arange(1,T+1):
            Pea[t]=np.exp(bet[0]+bet[1]*np.log(zt[t])+bet[2]*np.log(k[t-1]))
            c[t]=1/(beta*Pea[t])
            
            func = lambda x: a*c[t]-(1-alf)*zt[t]*k[t-1]**alf*x**(-alf)*(1-x)
            l[t] = opt.root_scalar(func, bracket=[lmin,lmax]).root
            if l[t] < 0:
                l[t] = 0
            elif l[t] > 1:
                l[t] = 1
            
            y[t] = zt[t]*k[t-1]**alf*l[t]**(1-alf)
            k[t] = zt[t]*k[t-1]**alf*l[t]**(1-alf)+(1-delta)*k[t-1]-c[t]
            inv[t] = y[t]-c[t]
        

        E3=((c[2:T+1]**(-1))*(alf*zt[2:T+1]*((k[1:T]**(alf-1))*(l[2:T+1]**(1-alf)))+(1-delta)))

        # calculating the minimizing parameters
        bet = nlls(E3,np.log(zt[1:T]),np.log(k[0:T-1]),T,bet0)
        eva = np.array([np.mean(k), np.mean(c), np.mean(l), np.mean(y), np.mean(inv)])
        print("Means:", eva)
        print("iteration = ", it, ", error = ", np.abs(bet-bet0).max(), sep='')
    
    
    stop = time.perf_counter()
    print("Elapsed time for the while loop is", round(stop - start,4), "seconds")
    
    with open('parameters.npy', 'wb') as f:
        np.save(f, bet)


Means: [260.8931   0.2986   0.8797   6.8456   6.547 ]
iteration = 1, error = 0.03285409926328264
Means: [260.5185   0.3005   0.8789   6.8381   6.5376]
iteration = 2, error = 0.03254591440995025
Means: [260.1416   0.3024   0.8781   6.8306   6.5281]
iteration = 3, error = 0.03223827461446782
Means: [259.7625   0.3044   0.8773   6.823    6.5186]
iteration = 4, error = 0.03193119841970726
Means: [259.381    0.3063   0.8765   6.8154   6.509 ]
iteration = 5, error = 0.03162470422981212
Means: [258.9971   0.3083   0.8757   6.8077   6.4994]
iteration = 6, error = 0.031318810319064916
Means: [258.6109   0.3103   0.8748   6.8      6.4897]
iteration = 7, error = 0.031013534814299692
Means: [258.2224   0.3122   0.874    6.7922   6.48  ]
iteration = 8, error = 0.030708895694592453
Means: [257.8315   0.3142   0.8732   6.7844   6.4702]
iteration = 9, error = 0.030404910795466478
Means: [257.4382   0.3163   0.8724   6.7765   6.4603]
iteration = 10, error = 0.030101597794276014
Means: [257.0425   0.318

Means: [220.4164   0.501    0.7936   6.0321   5.5311]
iteration = 85, error = 0.010300829093982111
Means: [219.8136   0.5039   0.7923   6.0199   5.516 ]
iteration = 86, error = 0.010086237697288114
Means: [219.2078   0.5069   0.791    6.0076   5.5008]
iteration = 87, error = 0.009873120136420388
Means: [218.599    0.5098   0.7897   5.9953   5.4855]
iteration = 88, error = 0.009661479925331173
Means: [217.9871   0.5128   0.7884   5.9829   5.4701]
iteration = 89, error = 0.009451320441858924
Means: [217.3721   0.5158   0.7871   5.9705   5.4547]
iteration = 90, error = 0.0092426449213312
Means: [216.7541   0.5188   0.7858   5.9579   5.4392]
iteration = 91, error = 0.009035456472267223
Means: [216.133    0.5218   0.7845   5.9454   5.4236]
iteration = 92, error = 0.008829758064798465
Means: [215.5088   0.5248   0.7831   5.9327   5.4079]
iteration = 93, error = 0.00862555253980668
Means: [214.8815   0.5278   0.7818   5.92     5.3922]
iteration = 94, error = 0.00842284261238957
Means: [214.25

Means: [160.1144   0.7773   0.6635   4.7949   4.0176]
iteration = 168, error = 0.004337757346456861
Means: [159.2714   0.7808   0.6616   4.7773   3.9964]
iteration = 169, error = 0.0043211905127504835
Means: [158.4261   0.7844   0.6598   4.7596   3.9752]
iteration = 170, error = 0.004304873221872618
Means: [157.5787   0.788    0.6579   4.7419   3.9539]
iteration = 171, error = 0.004288805139699592
Means: [156.7291   0.7915   0.6561   4.7241   3.9326]
iteration = 172, error = 0.004272985927667339
Means: [155.8773   0.7951   0.6542   4.7063   3.9112]
iteration = 173, error = 0.004257415240535689
Means: [155.0235   0.7986   0.6524   4.6884   3.8898]
iteration = 174, error = 0.004242092728000135
Means: [154.1675   0.8022   0.6505   4.6705   3.8683]
iteration = 175, error = 0.004227018034856589
Means: [153.3096   0.8057   0.6486   4.6525   3.8468]
iteration = 176, error = 0.004212190799117721
Means: [152.4496   0.8092   0.6467   4.6344   3.8252]
iteration = 177, error = 0.004197610653291828

Means: [86.947   1.0287  0.4994  3.2098  2.1812]
iteration = 251, error = 0.00504640765632125
Means: [86.1009  1.0305  0.4975  3.1905  2.1599]
iteration = 252, error = 0.005024692986879575
Means: [85.2578  1.0324  0.4955  3.1712  2.1388]
iteration = 253, error = 0.00500187037537092
Means: [84.4179  1.0342  0.4935  3.1519  2.1177]
iteration = 254, error = 0.004977949783572466
Means: [83.5813  1.036   0.4916  3.1327  2.0967]
iteration = 255, error = 0.004952941413677303
Means: [82.748   1.0377  0.4896  3.1135  2.0758]
iteration = 256, error = 0.004926855714717737
Means: [81.9182  1.0394  0.4877  3.0943  2.0549]
iteration = 257, error = 0.0048997033749100805
Means: [81.0919  1.041   0.4858  3.0752  2.0342]
iteration = 258, error = 0.004871495355277311
Means: [80.2691  1.0426  0.4838  3.0561  2.0136]
iteration = 259, error = 0.004842242852102885
Means: [79.4501  1.0441  0.4819  3.0371  1.993 ]
iteration = 260, error = 0.004811957325650429
Means: [78.6349  1.0456  0.48    3.0182  1.9725]
it

Means: [32.2206  1.0259  0.3637  1.8335  0.8076]
iteration = 338, error = 0.0071891708156114065
Means: [31.8581  1.0244  0.3627  1.8228  0.7985]
iteration = 339, error = 0.007245717484372194
Means: [31.5013  1.0228  0.3617  1.8123  0.7895]
iteration = 340, error = 0.007301605155593999
Means: [31.1502  1.0212  0.3607  1.8019  0.7807]
iteration = 341, error = 0.007356791459220954
Means: [30.8048  1.0196  0.3598  1.7917  0.7721]
iteration = 342, error = 0.007411234387409538
Means: [30.465   1.018   0.3588  1.7815  0.7635]
iteration = 343, error = 0.007464892378179835
Means: [30.1308  1.0164  0.3579  1.7715  0.7551]
iteration = 344, error = 0.007517724382795826
Means: [29.802   1.0148  0.357   1.7617  0.7469]
iteration = 345, error = 0.0075696899449961275
Means: [29.4787  1.0132  0.3561  1.7519  0.7388]
iteration = 346, error = 0.007620749272059482
Means: [29.1608  1.0115  0.3552  1.7423  0.7308]
iteration = 347, error = 0.007670863294188646
Means: [28.8482  1.0099  0.3543  1.7328  0.7229]

Means: [15.9801  0.9018  0.3159  1.3018  0.4   ]
iteration = 425, error = 0.007378281013975557
Means: [15.9053  0.9008  0.3156  1.2989  0.3981]
iteration = 426, error = 0.0073269100274623855
Means: [15.832   0.8999  0.3154  1.2962  0.3963]
iteration = 427, error = 0.007274885990883606
Means: [15.7599  0.899   0.3152  1.2934  0.3944]
iteration = 428, error = 0.007222236113613112
Means: [15.6891  0.898   0.3149  1.2907  0.3927]
iteration = 429, error = 0.007168987512233549
Means: [15.6196  0.8971  0.3147  1.2881  0.3909]
iteration = 430, error = 0.007115167220099139
Means: [15.5513  0.8962  0.3145  1.2855  0.3892]
iteration = 431, error = 0.007060802128928956
Means: [15.4842  0.8954  0.3143  1.2829  0.3875]
iteration = 432, error = 0.007005919004911987
Means: [15.4183  0.8945  0.3141  1.2804  0.3859]
iteration = 433, error = 0.0069505444446152365
Means: [15.3536  0.8936  0.3139  1.2779  0.3843]
iteration = 434, error = 0.006894704874506552
Means: [15.2901  0.8928  0.3136  1.2754  0.3827]

Means: [12.6183  0.8529  0.3047  1.1685  0.3156]
iteration = 511, error = 0.0027385813340238263
Means: [12.6009  0.8526  0.3047  1.1677  0.3152]
iteration = 512, error = 0.0026983093406138248
Means: [12.5838  0.8523  0.3046  1.167   0.3147]
iteration = 513, error = 0.002658503156746428
Means: [12.5669  0.852   0.3045  1.1663  0.3143]
iteration = 514, error = 0.002619161716146845
Means: [12.5503  0.8517  0.3045  1.1656  0.3139]
iteration = 515, error = 0.0025802838027313424
Means: [12.534   0.8514  0.3044  1.1649  0.3135]
iteration = 516, error = 0.0025418680552837247
Means: [12.518   0.8512  0.3044  1.1643  0.3131]
iteration = 517, error = 0.002503912929948282
Means: [12.5022  0.8509  0.3043  1.1636  0.3127]
iteration = 518, error = 0.0024664167698170147
Means: [12.4866  0.8507  0.3043  1.1629  0.3123]
iteration = 519, error = 0.0024293777572402253
Means: [12.4713  0.8504  0.3042  1.1623  0.3119]
iteration = 520, error = 0.0023927939468689208
Means: [12.4562  0.8501  0.3042  1.1617  0.

Means: [11.7937  0.8386  0.3019  1.1335  0.2949]
iteration = 597, error = 0.000686538339624454
Means: [11.7892  0.8385  0.3019  1.1333  0.2948]
iteration = 598, error = 0.0006751103387387936
Means: [11.7848  0.8385  0.3018  1.1331  0.2947]
iteration = 599, error = 0.0006638688884622601
Means: [11.7804  0.8384  0.3018  1.133   0.2946]
iteration = 600, error = 0.0006528111578967821
Means: [11.7761  0.8383  0.3018  1.1328  0.2945]
iteration = 601, error = 0.000641934375439468
Means: [11.7718  0.8382  0.3018  1.1326  0.2944]
iteration = 602, error = 0.0006312357845572603
Means: [11.7676  0.8382  0.3018  1.1324  0.2942]
iteration = 603, error = 0.0006207126611513791
Means: [11.7635  0.8381  0.3018  1.1322  0.2941]
iteration = 604, error = 0.0006103623205995223
Means: [11.7595  0.838   0.3018  1.1321  0.294 ]
iteration = 605, error = 0.0006001821031194621
Means: [11.7555  0.8379  0.3017  1.1319  0.2939]
iteration = 606, error = 0.000590169394295903
Means: [11.7516  0.8379  0.3017  1.1317  0.

Means: [11.577   0.8347  0.3011  1.1242  0.2895]
iteration = 683, error = 0.0001618004629500014
Means: [11.5758  0.8347  0.3011  1.1241  0.2894]
iteration = 684, error = 0.00015912531233625016
Means: [11.5746  0.8347  0.3011  1.1241  0.2894]
iteration = 685, error = 0.00015649522980737407
Means: [11.5734  0.8346  0.3011  1.124   0.2894]
iteration = 686, error = 0.00015390944563781206
Means: [11.5723  0.8346  0.3011  1.124   0.2893]
iteration = 687, error = 0.00015136719750413752
Means: [11.5711  0.8346  0.3011  1.1239  0.2893]
iteration = 688, error = 0.0001488677556639728
Means: [11.57    0.8346  0.3011  1.1239  0.2893]
iteration = 689, error = 0.00014641037961399261
Means: [11.5689  0.8346  0.3011  1.1238  0.2893]
iteration = 690, error = 0.00014399436300555069
Means: [11.5678  0.8345  0.3011  1.1238  0.2892]
iteration = 691, error = 0.00014161900121478777
Means: [11.5668  0.8345  0.3011  1.1237  0.2892]
iteration = 692, error = 0.0001392835936475656
Means: [11.5657  0.8345  0.3011  

In [8]:
print(bet)

[[ 1.5894]
 [-0.393 ]
 [-0.571 ]]


In [9]:
##############
# Simulation
#############

if siyes:
    T = 115
    N = 100
    ss = np.zeros((N,6))
    cc = np.zeros((N,6))
    
    for j in np.arange(0,N):
        r = rng.standard_normal((T+1,1))
        zt = np.ones((T+1,1))
        zt[0] = 1
        kt = np.zeros((T+1,1))
        kt[0] = k_ss
        it = np.zeros((T,1))
        ct = np.zeros((T,1))
        yt = np.zeros((T,1))
        lt = np.zeros((T,1))
        prodt = np.zeros((T,1))
        Peat = np.zeros((T,1))
        for t in np.arange(0,T):
            zt[t+1] = np.exp(rho* np.log(zt[t]) + sigmae*r[t])
            Peat[t] = np.exp(bet[0]+bet[1]*np.log(zt[t])+bet[2]*np.log(kt[t]))
            ct[t] = 1/(beta*Peat[t])
            func = lambda x: a*ct[t]-(1-alf)*zt[t]*kt[t]**alf*x**(-alf)*(1-x)
            lt[t] = opt.root_scalar(func, bracket=[lmin,lmax]).root
            if lt[t] < 0:
                lt[t] = 0
            elif lt[t] > 1:
                lt[t] = 1
            
            yt[t] = zt[t]*kt[t]**alf*lt[t]**(1-alf)
            kt[t+1] = zt[t]*kt[t]**alf*lt[t]**(1-alf)+(1-delta)*kt[t]-ct[t]
            it[t]= yt[t]-ct[t]
            prodt[t] = yt[t]/lt[t]
            
            
        z = zt[0:T]
        k = np.log(kt[0:T])
        y = np.log(yt[0:T])
        c = np.log(ct)
        i = np.log(it)
        h = np.log(lt[0:T])
        prod = np.log(prodt)
        dhp, dtr = hp1(np.concatenate((y, i, c, k, h, prod),axis=1),1600)
        ss[j,:] = np.std(dhp,axis=0,ddof=1)*100
        Corr = np.corrcoef(dhp,rowvar=False)
        cc[j,:] = Corr[:,0]
        
    std = np.mean(ss,axis=0)
    corr = np.mean(cc,axis=0)
    
    print('HANSEN: std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:')
    print(np.concatenate((np.array([[1.36, 4.24, 0.42, 0.36, 0.7, 0.68]]).T/1.36, \
                          np.array([[1, 0.99, 0.89, 0.06, 0.98, 0.98]]).T),axis=1))
    print('std(x) std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:')
    print(np.concatenate((std[:,np.newaxis], (std/std[0])[:,np.newaxis], \
                          corr[:,np.newaxis]),axis=1))
        


HANSEN: std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:
[[1.     1.    ]
 [3.1176 0.99  ]
 [0.3088 0.89  ]
 [0.2647 0.06  ]
 [0.5147 0.98  ]
 [0.5    0.98  ]]
std(x) std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:
[[1.3614 1.     1.    ]
 [4.2715 3.1375 0.9903]
 [0.4221 0.31   0.8909]
 [0.3711 0.2726 0.0758]
 [0.7027 0.5161 0.9813]
 [0.6856 0.5036 0.9805]]


In [10]:
#!jupyter nbconvert --to script hansen_PEA.ipynb